In [1]:
import torch
import cv2

In [2]:
from dataset import DRSegmentationDataset

In [16]:
test_dataset = DRSegmentationDataset("/Users/jakub/PG/collaborative_learning/diabetic_retinopathy/datasets/processed_segmentation_dataset/test_set")
train_dataset = DRSegmentationDataset("/Users/jakub/PG/collaborative_learning/diabetic_retinopathy/datasets/processed_segmentation_dataset/train_set")

In [17]:
test_dataset_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0,
)

train_dataset_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0,
)

In [18]:
input, target = next(iter(train_dataset_loader))

torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])
torch.Size([5, 640, 640])


# Show example image

In [19]:
cv2.imshow("input", input[1].numpy().transpose(1, 2, 0))
cv2.imshow("target", target[1][1, ...].numpy())
cv2.waitKey(0)
cv2.destroyAllWindows()